In [1]:
import torch
import cv2
import numpy as np

In [2]:
dir = "model.pth"
loadedModel = torch.load(dir, map_location=torch.device('cpu'))
print(loadedModel) # Print out model details.

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# def pred(y_hat): # OUR PRED FUNCTION.
#     print(y_hat)
#     human = torch.tensor([0,1])
#     dog = torch.tensor([1,0])
#     both = dog+human
#     if(torch.all(y_hat.round().eq(human))):
#         return "A human"
#     elif(torch.all(y_hat.round().eq(dog))):
#         return "A dog"
#     elif(torch.all(y_hat.round().eq(both))):
#         return "A human, a dog"
#     else: # catch all case; i.e. [0,0]
#         return "Background"

def pred(y_hat):
    print(y_hat)
    y_hat = torch.round(y_hat)
    classes_in_frame = ""
    classes = ["a dog", "a human", "a pen"]
    for i in range(y_hat.shape[1]):
        if(y_hat[0][i].item() == 1):
            classes_in_frame += classes[i] + ", "
    
    if(not classes_in_frame): # If no classes were found
        classes_in_frame = "Background"
    else: # If any classes were found
        classes_in_frame = classes_in_frame[:-2] # Remove the final comma
        
    return classes_in_frame
            
            

In [1]:
# INFERENCE

import torchvision.transforms as transforms
from skimage import io
from torch.utils.data import Dataset
class LoadData(Dataset):
    def __init__(self, image, transform=None):
        self.transform = transform
        self.image = image
    
    def __len__(self):
        return 1
    
    def __getitem__(self, index):
        image = self.image

        if(len(image.shape) == 2 or image.shape[2] == 1): # Convert to rgb if grayscale img 
          image = gray2rgb(io.imread(img_path))

        if(image.shape[2] == 4): # Convert to rgb if rgba img 
          image = rgba2rgb(io.imread(img_path))
        
        # Apply the transforms, if any 
        if self.transform:
            image = self.transform(image.copy())
        return (image, [[]])
    
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224,224)), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
                                transforms.RandomCrop(224, padding=10, padding_mode='reflect'), transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip()])

cap = cv2.VideoCapture(0)


while True: 
    ret, frame = cap.read()
    cv2.imwrite("frame.jpg", frame)
    width = int(cap.get(3))
    height = int(cap.get(4))
    font = cv2.FONT_ITALIC
    loadedModel.eval() # set to eval mode to make BatchNorm behave normally

    ds = LoadData(frame, transform=transform)
    dl = torch.utils.data.DataLoader(dataset=ds, batch_size=1)
    y_hat_inf = torch.tensor([[]])
    for (images, labels) in dl:
        y_hat_inf = loadedModel(images)
    image = cv2.putText(frame, pred(y_hat_inf), (width-1100, height-15), font, 1, (255,255,255), 1, cv2.LINE_AA)
    cv2.imshow(" FRAME", frame)
    if(cv2.waitKey(1) == ord('q')):
        break
    
cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined